In [3]:
import numpy as np 
import pandas as pd 
import torch 
import torch.nn as nn
import os
import PIL.Image as Image
import kagglehub


In [3]:
%pip install kagglehub


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:

path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")

print("Path to dataset files:", path)


Path to dataset files: /Users/advaykadam/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

train_data_dir = os.path.join(path, 'real_vs_fake/real-vs-fake/train')

test_data_dir = os.path.join(path, 'real_vs_fake/real-vs-fake/test')

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255)

target_size = (224, 224)
batch_size = 100  

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size= target_size, 
    batch_size=batch_size,
    class_mode='binary', 
    shuffle=True
)

Found 100000 images belonging to 2 classes.


In [8]:
train_generator.class_indices

{'fake': 0, 'real': 1}

In [44]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers.legacy import Adam  

def build_model():
    shape = (224, 224, 3)
    
    Xception_mod_trained = Xception(input_shape=shape, include_top=False, weights='imagenet')  
    
    
    final_model_transfer = Sequential()
    
    final_model_transfer.add(Xception_mod_trained)
    
    final_model_transfer.add(layers.GlobalAveragePooling2D())
    final_model_transfer.add(layers.Dense(1024, activation='relu'))
    final_model_transfer.add(layers.Dropout(0.5))
    
    final_model_transfer.add(layers.Dense(1, activation='sigmoid'))  
    
    final_model_transfer.compile(optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'], loss = 'binary_crossentropy')


    return final_model_transfer

In [45]:
final_model_transfer.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 22960681 (87.59 MB)
Trainable params: 22906153 (87.38 MB)
Non-trainable params: 54528 (213.00 KB)
__________

In [46]:
from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint_callback = ModelCheckpoint(
    'model_checkpoint_{epoch:03d}.h5',  
    save_weights_only=True,  
    save_freq='epoch', 
    verbose=1
)

In [47]:
import numpy as np

def train_in_chunks(train_generator, model, batch_size=100, chunk_size=10000, checkpoint_callback=None):

    total_samples = train_generator.samples 
    
    steps_per_chunk = chunk_size // batch_size  
    
    for start_idx in range(0, total_samples, chunk_size):

        end_idx = min(start_idx + chunk_size, total_samples)
        
     
        model.fit(
            train_generator,
            steps_per_epoch=steps_per_chunk,
            epochs=1, 
            callbacks=[checkpoint_callback]
        )

In [48]:
tran_model = build_model()

In [49]:
train_in_chunks(train_generator, tran_model, chunk_size=10000, batch_size=100, checkpoint_callback=checkpoint_callback)

100/100 [==============================] - ETA: 0s - loss: 0.6239 - accuracy: 0.6622 
Epoch 1: saving model to model_checkpoint_001.h5
100/100 [==============================] - ETA: 0s - loss: 0.4664 - accuracy: 0.7951 
Epoch 1: saving model to model_checkpoint_001.h5
100/100 [==============================] - ETA: 0s - loss: 0.3407 - accuracy: 0.8602 
Epoch 1: saving model to model_checkpoint_001.h5
100/100 [==============================] - ETA: 0s - loss: 0.2606 - accuracy: 0.8940 
Epoch 1: saving model to model_checkpoint_001.h5
100/100 [==============================] - ETA: 0s - loss: 0.2040 - accuracy: 0.9202 
Epoch 1: saving model to model_checkpoint_001.h5
100/100 [==============================] - ETA: 0s - loss: 0.1713 - accuracy: 0.9337 
Epoch 1: saving model to model_checkpoint_001.h5
100/100 [==============================] - ETA: 0s - loss: 0.1466 - accuracy: 0.9453 
Epoch 1: saving model to model_checkpoint_001.h5
100/100 [==============================] - ETA: 0s - lo

Test Data Generation

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)

target_size = (224, 224)
batch_size = 100  

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size= target_size, 
    batch_size=batch_size,
    class_mode='binary', 
    shuffle=True
)

Found 20000 images belonging to 2 classes.


In [11]:
test_generator.class_indices

{'fake': 0, 'real': 1}

In [51]:
tran_model.evaluate(test_generator)

200/200 [==============================] - 536s 3s/step - loss: 0.1034 - accuracy: 0.9607


[0.10337487608194351, 0.9606500267982483]

Validation

In [ ]:
tran_model.save("transfer_model.keras")